# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Spark: Structured Streaming (Kafka consumer application)** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [7]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

### Creación del Kafka Stream

In [9]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "cfb7d55576f2:9093") \
                .option("subscribe", "kafka-spark-example") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transform binary data into string

In [10]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

### Configuración del "Sink" del stream

In [11]:

query = kafka_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='3 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(300)


25/04/10 01:36:26 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-695930a1-72a1-4909-9720-2014efbbdd2f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/10 01:36:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/10 01:36:26 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+-----+---------+------+---------+-------------+---------+
|key|value|topic|partition|offset|timestamp|timestampType|value_str|
+---+-----+-----+---------+------+---------+-------------+---------+
+---+-----+-----+---------+------+---------+-------------+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                                                                                                                                             |topic              |partition|offset|timestamp              |timestampType|value_str                                                                                  |
+----

-------------------------------------------
Batch: 2
-------------------------------------------
+----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+--------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                                                                                                                                                |topic              |partition|offset|timestamp              |timestampType|value_str                                                                                  

-------------------------------------------
Batch: 6
-------------------------------------------
+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                                                                                                                                             |topic              |partition|offset|timestamp              |timestampType|value_str                                                                                  |
+----

...

-------------------------------------------
Batch: 13
-------------------------------------------
+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                                                                                                                                             |topic              |partition|offset|timestamp              |timestampType|value_str                                                                                  |
+---

-------------------------------------------
Batch: 16
-------------------------------------------
+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                                                                                                                                             |topic              |partition|offset|timestamp              |timestampType|value_str                                                                                  |
+---

-------------------------------------------
Batch: 19
-------------------------------------------
+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                                                                                                                                             |topic              |partition|offset|timestamp              |timestampType|value_str                                                                                  |
+---

-------------------------------------------
Batch: 20
-------------------------------------------
+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+---------+------+-----------------------+-------------+-------------------------------------------------------------------------------------------+
|key |value                                                                                                                                                                                                                                                                             |topic              |partition|offset|timestamp              |timestampType|value_str                                                                                  |
+---

False

In [ ]:
#query.stop()
#sc.stop()

25/04/10 02:04:09 ERROR TaskSchedulerImpl: Lost executor 1 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/04/10 02:04:09 ERROR TaskSchedulerImpl: Lost executor 0 on 172.24.0.5: worker lost: Not receiving heartbeat for 60 seconds
25/04/10 03:09:30 ERROR TaskSchedulerImpl: Lost executor 3 on 172.24.0.5: worker lost: Not receiving heartbeat for 60 seconds
25/04/10 03:09:30 ERROR TaskSchedulerImpl: Lost executor 2 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/04/10 03:46:21 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 2184299 milliseconds
25/04/10 03:57:42 ERROR TaskSchedulerImpl: Lost executor 5 on 172.24.0.4: worker lost: Not receiving heartbeat for 60 seconds
25/04/10 03:57:42 ERROR TaskSchedulerImpl: Lost executor 4 on 172.24.0.5: worker lost: Not receiving heartbeat for 60 seconds
25/04/10 04:07:38 ERROR TaskSchedulerImpl: Lost executor 7 on 172.24.0.5: worker lost: Not